In [1]:
import os
os.getcwd()
os.chdir('/home/siyuan/Seafile/Thesis/PycharmProjects/DD_PE_paper/')

In [2]:
from qiskit.transpiler import PassManager, InstructionDurations
from src.tools.DD_insertion import construct_bv_circuit, \
                                 construct_graph_matrix, \
                                 convert_count_to_prob, \
                                 translate_circuit_to_basis
from qiskit.circuit.library import QFT, GraphState

In [3]:
from qiskit import IBMQ, transpile, Aer
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default')
backend = provider.get_backend('ibmq_toronto')

In [4]:
bv_circuits = []
hs_circuits = []
qft_circuits = []
for i in range(3, 15):
    bv_circuits.append(construct_bv_circuit(i))

# for i in range(2, 15, 2):
#     hs_circuits.append(construct_hs_circuit(i))

for i in range(3, 15):
    qft_circuits.append(QFT(i))

for circuit in bv_circuits:
    circuit.measure_all()

# for circuit in hs_circuits:
#     circuit.measure_all()

for circuit in qft_circuits:
    circuit.measure_all()

In [5]:
durations = InstructionDurations.from_backend(backend)
## add duration of y gates which are used for DD sequences
bconf = backend.configuration()
for i in range(bconf.num_qubits):
    x_duration = durations.get('x', i)
    durations.update(InstructionDurations(
        [('y', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('rx', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('ry', i, x_duration)]
        ))

In [6]:
graph_state_circuits = []
coupling_map = backend.configuration().coupling_map

for i in range(3, 15):
    gs_circuit_matrix = construct_graph_matrix(i, coupling_map)
    graph_state_circuits.append(GraphState(gs_circuit_matrix))

for circuit in graph_state_circuits:
    circuit.measure_all()

In [ ]:
from src.tools.DD_insertion import pm_DD_sequences
pms = pm_DD_sequences(durations)

In [ ]:
bv_job_ids = []
bv_jobs = []

for circuit in bv_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    bv_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    bv_job_ids.append(job_id)

In [ ]:
for job_id in bv_job_ids:
    print(job_id)

In [7]:
from qiskit import execute

shots = 8192
psts = []
bv_job_ids = [
    '628e23ca9fca8f6c12716afa',
    '628e23d49fca8fcfdf716afb',
    '628e23d641fd145a3fa417a5',
    '628e23daade1491273def42e',
    '628e23e09fca8f6fca716afc',
    '628e23e404d374d5d065d6d5',
    '628e23e8fdc45291f1b2b950',
    '628e23ecc7a7355bfd31878d',
    '628e23f0fdc4525d86b2b951',
    '628e23f404d374432f65d6d7',
    '628e23f938224a0150f085a7',
    '628e23fdf559705069d5c6f2',

]
for i, circuit in enumerate(bv_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    ideal_result = list(ideal_counts.keys())[0]
    result_counts = backend.retrieve_job(bv_job_ids[i]).result().get_counts()
    pst = []
    for result_count in result_counts:
        pst.append(result_count[ideal_result]/shots)
    psts.append(pst)

bv_circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', 'BV13', 'BV14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']
bv_results_data = {}

for i, name in enumerate(bv_circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, pst in enumerate(psts[i]):
        print(f'pst result of {DD_sequences[j]}: {pst}')
    bv_results_data[name] = psts[i]

---------Result of circuit BV3 -------------
pst result of no_dd: 0.4266357421875
pst result of hahn_X: 0.427978515625
pst result of hahn_Y: 0.4208984375
pst result of CP: 0.435302734375
pst result of CPMG: 0.42578125
pst result of xy4: 0.4149169921875
pst result of xy8: 0.4139404296875
pst result of xy16: 0.4150390625
pst result of udd1: 0.4200439453125
pst result of udd2: 0.4232177734375
pst result of kdd: 0.4234619140625
---------Result of circuit BV4 -------------
pst result of no_dd: 0.40966796875
pst result of hahn_X: 0.394775390625
pst result of hahn_Y: 0.4029541015625
pst result of CP: 0.41552734375
pst result of CPMG: 0.4156494140625
pst result of xy4: 0.4053955078125
pst result of xy8: 0.415283203125
pst result of xy16: 0.404541015625
pst result of udd1: 0.409912109375
pst result of udd2: 0.4141845703125
pst result of kdd: 0.411865234375
---------Result of circuit BV5 -------------
pst result of no_dd: 0.247314453125
pst result of hahn_X: 0.2344970703125
pst result of hahn_Y:

In [8]:
bv_results_data

{'BV3': [0.4266357421875,
  0.427978515625,
  0.4208984375,
  0.435302734375,
  0.42578125,
  0.4149169921875,
  0.4139404296875,
  0.4150390625,
  0.4200439453125,
  0.4232177734375,
  0.4234619140625],
 'BV4': [0.40966796875,
  0.394775390625,
  0.4029541015625,
  0.41552734375,
  0.4156494140625,
  0.4053955078125,
  0.415283203125,
  0.404541015625,
  0.409912109375,
  0.4141845703125,
  0.411865234375],
 'BV5': [0.247314453125,
  0.2344970703125,
  0.2459716796875,
  0.2581787109375,
  0.2430419921875,
  0.265380859375,
  0.25,
  0.2357177734375,
  0.2421875,
  0.2049560546875,
  0.2176513671875],
 'BV6': [0.0809326171875,
  0.0782470703125,
  0.0797119140625,
  0.1470947265625,
  0.1490478515625,
  0.151123046875,
  0.222900390625,
  0.158935546875,
  0.208984375,
  0.228271484375,
  0.1864013671875],
 'BV7': [0.019775390625,
  0.013671875,
  0.0167236328125,
  0.0899658203125,
  0.0977783203125,
  0.1015625,
  0.1300048828125,
  0.08740234375,
  0.1328125,
  0.129638671875,
  0.

In [9]:
# import numpy as np
# import matplotlib.pyplot as plt

# baseline = []
# data = []
# for name in bv_circuit_names:
#     result_data = bv_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [i / result_data[0] for i in result_data[1:]]
#     data.append(ratio)

# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[1:, 2]
# cpmg = data[1:, 3]
# xy4 = data[1:, 4]
# xy8 = data[1:, 5]
# xy16 = list(data[2:, 6])
# xy16.insert(0, None)
# udd_X = data[1:, 7]
# udd_Y = data[1:, 8]
# kdd = list(data[2:, 9])
# kdd.insert(0, None)

# circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', ]

# x_ticks = circuit_names[:6]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure

# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0, 0, 1, 1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2][:6], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2][:6], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2][:6], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2][:6], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2][:6], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2][:6], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2][:6], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2][:6], linestyle='--', marker='*', color='m', label='kdd')

# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative PST result for BV circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative PST', fontsize=16)
# plt.savefig('TQE_Relat_PST_BV_toronto.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
gs_job_ids = []
gs_jobs = []

for circuit in graph_state_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    gs_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    gs_job_ids.append(job_id)

In [ ]:
for job_id in gs_job_ids:
    print(job_id)

In [10]:
from scipy.spatial.distance import jensenshannon
from qiskit import execute
gs_jsds = []

gs_job_ids = [
    '628e23ff732c3956d3d6824d',
    '628e240138224a10b1f085a8',
    '628e240304d374092d65d6d9',
    '628e240441fd148511a417a8',
    '628e240638224a4e74f085a9',
    '628e240804d374327b65d6da',
    '628e240a9fca8fb7e0716afe',
    '628e241104d374201b65d6db',
    '628e24149fca8f33c7716aff',
    '628e241604d374196f65d6dc',
    '628e2418fdc452c2aeb2b954',
    '628e241a04d374339865d6dd',
]


shots = 8192

for i, circuit in enumerate(graph_state_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(gs_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    gs_jsds.append(jsd)

circuit_names = ['GS3', 'GS4', 'GS5', 'GS6', 'GS7', 'GS8', 'GS9', 'GS10', 'GS11', 'GS12', 'GS13', 'GS14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

gs_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(gs_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    gs_results_data[name] = gs_jsds[i]

---------Result of circuit GS3 -------------
jsd result of no_dd: 0.1780871582090334
jsd result of hahn_X: 0.17661840240614488
jsd result of hahn_Y: 0.17357326983962465
jsd result of CP: 0.16200706613558977
jsd result of CPMG: 0.16303289361730533
jsd result of xy4: 0.1713722274521979
jsd result of xy8: 0.16563600658459507
jsd result of xy16: 0.16857544245164316
jsd result of udd1: 0.16281662625797536
jsd result of udd2: 0.1654866345788751
jsd result of kdd: 0.16941621101493134
---------Result of circuit GS4 -------------
jsd result of no_dd: 0.2084382295681628
jsd result of hahn_X: 0.20641147704411522
jsd result of hahn_Y: 0.2048389071215163
jsd result of CP: 0.1848656034645793
jsd result of CPMG: 0.18695966999635627
jsd result of xy4: 0.18883669260417538
jsd result of xy8: 0.19301585702217738
jsd result of xy16: 0.18001612577150378
jsd result of udd1: 0.18951477748074416
jsd result of udd2: 0.18626407039587492
jsd result of kdd: 0.1859818215068898
---------Result of circuit GS5 ------

In [11]:
gs_results_data

{'GS3': [0.1780871582090334,
  0.17661840240614488,
  0.17357326983962465,
  0.16200706613558977,
  0.16303289361730533,
  0.1713722274521979,
  0.16563600658459507,
  0.16857544245164316,
  0.16281662625797536,
  0.1654866345788751,
  0.16941621101493134],
 'GS4': [0.2084382295681628,
  0.20641147704411522,
  0.2048389071215163,
  0.1848656034645793,
  0.18695966999635627,
  0.18883669260417538,
  0.19301585702217738,
  0.18001612577150378,
  0.18951477748074416,
  0.18626407039587492,
  0.1859818215068898],
 'GS5': [0.19837933099116323,
  0.1960721760717116,
  0.1958795547034887,
  0.18011042852727344,
  0.1827239664397689,
  0.1939472326012384,
  0.17980561798990058,
  0.1778135834737276,
  0.18200183148047844,
  0.18899426762333474,
  0.17419554423020855],
 'GS6': [0.2790393425398427,
  0.27633179558743587,
  0.2711068266237297,
  0.2501255207793241,
  0.24001041959149497,
  0.23064721841620983,
  0.23372274783901592,
  0.2368083967897985,
  0.24062461449025013,
  0.234584465476247

In [12]:
baseline = []

data = []
for name in circuit_names:
    result_data = gs_results_data[name]
    baseline.append(result_data[0])
    ratio = [result_data[0]/i for i in result_data[1:]]
    data.append(ratio)


data = np.array(data)

# hahn_X = data[:, 0]
# hahn_Y = data[:, 1]
cp = data[:, 2]
cpmg = data[:, 3]
xy4 = data[:, 4]
xy8 = data[:, 5]
xy16 = data[:, 6]
udd_X = data[:, 7]
udd_Y = data[:, 8]
kdd = data[:, 9]

x_ticks = circuit_names[:-2]
X = np.arange(len(x_ticks))

from matplotlib.pyplot import figure
fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
ax = fig.add_axes([0,0,1,1])

# plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
plt.axhline(1, color='tab:brown', lw=2)
plt.legend(loc='best', fontsize=16)
# ax.set_title('Relative JSD result for GS circuit on ibmq_guadalupe', fontsize=18)
ax.set_xticks(X)
ax.set_xticklabels(x_ticks, fontsize=14)
ax.set_ylabel('Relative JSD', fontsize=16)

plt.savefig('TQE_Relat_JSD_GS_toronto.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
qft_job_ids = []
qft_jobs = []

for circuit in qft_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    qft_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    qft_job_ids.append(job_id)

In [ ]:
for job_id in qft_job_ids:
    print(job_id)

In [13]:
from scipy.spatial.distance import jensenshannon
qft_jsds = []

qft_job_ids = [
    '628e247ec7a7351d4e31878f',
    '628e248b732c395a1ad68250',
    '628e248fe2745b184cd266c9',
    '628e249afdc4520f0db2b956',
    '628e249e732c3905e1d68251',
    '628e24a438224ad237f085ab',
    '628e24ab04d374eb0a65d6e2',
    '628e24b2fdc452a4b1b2b957',
    '628e24bef559707ae9d5c6f3',
    '628e24cfade14930fbdef42f',
    '628e24dc9fca8f7ebf716b03',
    '628e24e938224a06ebf085ae'
]

shots = 8192

for i, circuit in enumerate(qft_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(qft_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    qft_jsds.append(jsd)

circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6', 'QFT7', 'QFT8', 'QFT9', 'QFT10', 'QFT11', 'QFT12', 'QFT13', 'QFT14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

qft_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(qft_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    qft_results_data[name] = qft_jsds[i]

---------Result of circuit QFT3 -------------
jsd result of no_dd: 0.10677248688666134
jsd result of hahn_X: 0.10408448064039981
jsd result of hahn_Y: 0.10139527644707166
jsd result of CP: 0.08806131881377216
jsd result of CPMG: 0.09373563322248972
jsd result of xy4: 0.09351352238814191
jsd result of xy8: 0.10903471681854089
jsd result of xy16: 0.09437134182246877
jsd result of udd1: 0.10037354985789994
jsd result of udd2: 0.08766327223711755
jsd result of kdd: 0.06501758156044514
---------Result of circuit QFT4 -------------
jsd result of no_dd: 0.06229069820574247
jsd result of hahn_X: 0.0648200465656519
jsd result of hahn_Y: 0.06585589694824574
jsd result of CP: 0.0561085799445222
jsd result of CPMG: 0.07913804228129856
jsd result of xy4: 0.061884089609272185
jsd result of xy8: 0.06341316637196973
jsd result of xy16: 0.06880879346673928
jsd result of udd1: 0.0878708636485954
jsd result of udd2: 0.06217648380284811
jsd result of kdd: 0.06010222547417728
---------Result of circuit QFT

In [16]:
qft_results_data

{'QFT3': [0.10677248688666134,
  0.10408448064039981,
  0.10139527644707166,
  0.08806131881377216,
  0.09373563322248972,
  0.09351352238814191,
  0.10903471681854089,
  0.09437134182246877,
  0.10037354985789994,
  0.08766327223711755,
  0.06501758156044514],
 'QFT4': [0.06229069820574247,
  0.0648200465656519,
  0.06585589694824574,
  0.0561085799445222,
  0.07913804228129856,
  0.061884089609272185,
  0.06341316637196973,
  0.06880879346673928,
  0.0878708636485954,
  0.06217648380284811,
  0.06010222547417728],
 'QFT5': [0.1880865294939582,
  0.18962770451405883,
  0.18797887404193836,
  0.15020194427402153,
  0.1462938113525112,
  0.14550499252949245,
  0.1715956247052801,
  0.15106114948001687,
  0.14476103270178353,
  0.1335692849301899,
  0.18142287761935663],
 'QFT6': [0.21181554919991524,
  0.21813231772842817,
  0.21724723330799967,
  0.1584479429508027,
  0.13175091991153687,
  0.1512278364125021,
  0.12240321028538428,
  0.14926735709764297,
  0.12275777949593679,
  0.111

In [15]:
# import matplotlib.pyplot as plt
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = qft_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for QFT circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_QFT_toronto.pdf', bbox_inches='tight', pad_inches=0)